In [27]:
import pandas as pd 
import numpy as np

In [28]:
df = pd.read_csv("taxi_trip_pricing.csv")
df

,Trip_Distance_km,Time_of_Day,Day_of_Week,Passenger_Count,Traffic_Conditions,Weather,Base_Fare,Per_Km_Rate,Per_Minute_Rate,Trip_Duration_Minutes,Trip_Price
0,19.35,Morning,Weekday,3.0,Low,Clear,3.56,0.80,0.32,53.82,36.2624
1,47.59,Afternoon,Weekday,1.0,High,Clear,NaN,0.62,0.43,40.57,NaN
2,36.87,Evening,Weekend,1.0,High,Clear,2.70,1.21,0.15,37.27,52.9032
3,30.33,Evening,Weekday,4.0,Low,NaN,3.48,0.51,0.15,116.81,36.4698
4,NaN,Evening,Weekday,3.0,High,Clear,2.93,0.63,0.32,22.64,15.6180
...,...,...,...,...,...,...,...,...,...,...,...
995,5.49,Afternoon,Weekend,4.0,Medium,Clear,2.39,0.62,0.49,58.39,34.4049
996,45.95,Night,Weekday,4.0,Medium,Clear,3.12,0.61,NaN,61.96,62.1295
997,7.70,Morning,Weekday,3.0,Low,Rain,2.08,1.78,NaN,54.18,33.1236
998,47.56,Morning,Weekday,1.0,Low,Clear,2.67,0.82,0.17,114.94,61.2090


In [29]:
df.isna().sum()

Trip_Distance_km         50
Time_of_Day              50
Day_of_Week              50
Passenger_Count          50
Traffic_Conditions       50
Weather                  50
Base_Fare                50
Per_Km_Rate              50
Per_Minute_Rate          50
Trip_Duration_Minutes    50
Trip_Price               49
dtype: int64

In [30]:
df.dropna(subset="Trip_Price",inplace=True)

In [31]:
df.isna().sum()

Trip_Distance_km         50
Time_of_Day              49
Day_of_Week              46
Passenger_Count          48
Traffic_Conditions       50
Weather                  46
Base_Fare                44
Per_Km_Rate              44
Per_Minute_Rate          49
Trip_Duration_Minutes    46
Trip_Price                0
dtype: int64

In [32]:
# from sklearn.impute import SimpleImputer
# char_imputer = SimpleImputer(strategy="most_frequent")
# num_imputer = SimpleImputer(strategy="median")

In [33]:
char_col = df.select_dtypes(include="object").columns
char_col

Index(['Time_of_Day', 'Day_of_Week', 'Traffic_Conditions', 'Weather'], dtype='object')

In [34]:
num_col = df.select_dtypes(include=np.number).columns.drop("Trip_Price")
num_col

Index(['Trip_Distance_km', 'Passenger_Count', 'Base_Fare', 'Per_Km_Rate',
       'Per_Minute_Rate', 'Trip_Duration_Minutes'],
      dtype='object')

In [35]:
x = df.iloc[:,:-1]
x
y = df.iloc[:,-1]
y

0      36.2624
2      52.9032
3      36.4698
4      15.6180
5      60.2028
        ...   
995    34.4049
996    62.1295
997    33.1236
998    61.2090
999    45.4437
Name: Trip_Price, Length: 951, dtype: float64

In [36]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2)

In [37]:
#Build Numeric Pipeline
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler   
from sklearn.preprocessing import OneHotEncoder 
numeric_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

In [38]:
char_pipeline = Pipeline([
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('ohn',OneHotEncoder(handle_unknown='ignore',sparse_output=False))
])

In [39]:
from sklearn.compose import ColumnTransformer
preproccess = ColumnTransformer([
    ('num',numeric_pipeline,num_col),
    ('char',char_pipeline,char_col)
])

In [40]:
from sklearn.linear_model import LinearRegression
final_pipe = Pipeline([
    ('preproccess',preproccess),
    ('model',LinearRegression())
])

In [41]:
final_pipe.fit(x_train,y_train)

,steps,"[('preproccess', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('char', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [44]:
y_pred = final_pipe.predict(x_test)
y_pred

array([ 80.25117187,  52.2024303 ,  15.6922513 ,  55.55948914,
        69.88017377,  53.33241124,  12.60590305,  71.26636165,
        82.13589824,  79.90612611,  27.06361122,  58.50126343,
         8.45303224,  41.25328185,  22.36146711,  46.83260573,
        71.50536084,  86.29288635,  85.0435826 ,  67.9043231 ,
         9.36606238,  75.28403649,  75.71908278,  18.7415956 ,
        54.18995172,  21.88283687,  29.63895664,  61.85092878,
        59.42907691,  29.67166506,  47.91553966,  22.22456435,
        29.03777091,  20.69230261,  47.2411599 ,  28.93713851,
        28.01483265,  81.93458533,  64.29337682,  11.35797696,
        53.94351312, -18.07131115, 124.60575091,   8.27216656,
        57.64502512,  36.87818962,  58.23342303,  16.26048924,
        70.03789034,  55.37183494,  58.71890513,  63.98303543,
        -6.19300395,   1.77861331,  81.31530856,   1.15771504,
        31.76285337,  22.96244591,   1.34445866,  36.50664212,
        31.11809726,  62.43879598,  61.97020496,  42.93

In [45]:
from sklearn.metrics import mean_absolute_percentage_error,r2_score
print(mean_absolute_percentage_error(y_test,y_pred))
print(r2_score(y_test,y_pred))

0.2926931390049378
0.8722096584839729
